<a href="https://colab.research.google.com/github/AjayBora002/movie_recommendation/blob/main/Copy_of_movie_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.Import Modules


In [ ]:
import pandas as pd
import numpy as np
import re
import nltk

In [ ]:
movies=pd.read_csv("https://raw.githubusercontent.com/AjayBora002/movie_recommendation/refs/heads/main/movies.csv")

In [ ]:


def clean_title(title):
    return re.sub(r'[^a-zA-Z0-9 ]', '', title.lower()).strip()


In [ ]:
movies["clean_title"]=movies["title"].apply(clean_title)# this will access a new clean title column


2. FINDING SIMILARITIES BETWEEN THE WORDS AND WITH THE HELP OF IT GIVING THE SIMILAR TYPE OF MOVIE TO THE USER

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer# converting text data into numerical data and finding similarities , this will do it
vectorizer = TfidfVectorizer(ngram_range=(1,2))# this ngram thing will help in finding instead of single single word it will match two consecutive words like toy story together
tfidf = vectorizer.fit_transform(movies["clean_title"])

3.finding similarity and creating search function

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(title):
    title = clean_title(title)  # clean the input title
    query_vec = vectorizer.transform([title])  # vectorize it
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    results = movies.iloc[indices].iloc[::-1]
    return results


4.CREATING AN INTERACTIVE BOX

In [ ]:
import ipywidgets as widgets
from IPython.display import display  # function used to show diiff things as output on notebook


movie_input = widgets.Text(    # creating a widget
    value='Toy Story',
    description = "Movie Title :",  # movie titled will be entered here
    disabled=False   # it will be enabled
)


# this whole thing will create a box to enter movie title , but wont show anything as output
  #NOW MAKING AN OUTPUT WIDGET
movie_list = widgets.Output()

def on_type(data):     # this fun will be called whenever we type something in the box
  with movie_list:
    movie_list.clear_output()

    title=data["new"]
    if len(title)>5:
      display(search(title))

movie_input.observe(on_type, names='value')  # there are diff events of the widgets whenever we input a movie name it is going to call on type and gives value event
display(movie_input, movie_list)




Text(value='Toy Story', description='Movie Title :')

Output()

In [ ]:
ratings=pd.read_csv("ratings.csv")

In [ ]:
ratings.dtypes

,0
userId,int64
movieId,int64
rating,int64
timestamp,int64


5.FINDING SIMILAR USER RECOMMENDATION

In [ ]:
movie_id=1

In [ ]:
similar_users = ratings[(ratings["movieId"] == movie_id)# this is for finding anyone who watched same movie
& (ratings["rating"] > 4)]["userId"].unique()# this is for anyone who watched the movie and gave the rating 5

similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
# this will show the movies that are rated by similar users and find their used ids

In [ ]:
similar_user_recs

,movieId
10,465
15,222
29,95
47,327
56,181
...,...
99896,97
99938,176
99953,93
99954,121


In [ ]:
similar_user_recs=similar_user_recs.value_counts()/len(similar_users) # this counts the no of 5 rates is given to a movie and then div by len shows recommending percent like top one has 100%

# we are doing this to find per of similar people who watched the movie
similar_user_recs = similar_user_recs[similar_user_recs > .1]  # shows movies having per greater than 10 %

In [ ]:
similar_user_recs

,count
movieId,
1,1.000000
50,0.588235
174,0.478992
318,0.403361
181,0.394958
...,...
150,0.100840
523,0.100840
272,0.100840


6.now problem is that most of the people like movies like toy story and avengers , what we need is how much movies are liked by users are we also like this will give more accuracy on recommendation

In [ ]:
all_users=ratings[(ratings)["movieId"].isin(similar_user_recs.index) & (ratings["rating"]>4)]

In [ ]:
all_users_recs=all_users["movieId"].value_counts()/len(all_users["userId"].unique())# this will give us percentage of all the users who liked the movies

In [ ]:
rec_percentages=pd.concat([similar_user_recs,all_users_recs],axis=1)# this concatenates the data of how much similar people to us liked and how much avg person liked it
rec_percentages.columns=["similar","all"]

In [ ]:
rec_percentages

,similar,all
movieId,,
1,1.000000,0.131929
50,0.588235,0.361419
174,0.478992,0.223947
318,0.403361,0.206208
181,0.394958,0.189579
...,...,...
150,0.100840,0.055432
523,0.100840,0.048780
272,0.100840,0.103104


7.NOW WE WANT MOVIES THAT HAS A BIG DIFF BET THESE TWO COLUMN

In [ ]:
rec_percentages["score"]=rec_percentages["similar"]/rec_percentages["all"]
rec_percentages=rec_percentages.sort_values("score",ascending=False)   # using pandas sort method to make an order of the score the higher the score more good will be the recommendation
rec_percentages

,similar,all,score
movieId,,,
1,1.000000,0.131929,7.579832
692,0.126050,0.029933,4.211018
378,0.117647,0.028825,4.081448
739,0.100840,0.025499,3.954695
71,0.252101,0.065410,3.854152
...,...,...,...
238,0.100840,0.086475,1.166128
269,0.117647,0.107539,1.093996
286,0.142857,0.145233,0.983642


8.NOW TAKING TOP 10 RATINGS AND MERGE IT WITH MOVIES DATA

In [ ]:
rec_percentages.head(10).merge(movies,left_index=True,right_on="movieId")

,similar,all,score,movieId,title,genres,clean_title
0,1.000000,0.131929,7.579832,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story 1995
677,0.126050,0.029933,4.211018,692,Solo (1996),Action|Sci-Fi|Thriller,solo 1996
373,0.117647,0.028825,4.081448,378,Speechless (1994),Comedy|Romance,speechless 1994
724,0.100840,0.025499,3.954695,739,Honey Moon (Honigmond) (1996),Comedy,honey moon honigmond 1996
70,0.252101,0.065410,3.854152,71,Fair Game (1995),Action,fair game 1995
187,0.100840,0.027716,3.638319,189,Reckless (1995),Comedy|Fantasy,reckless 1995
227,0.126050,0.035477,3.553046,230,Dolores Claiborne (1995),Drama|Thriller,dolores claiborne 1995
717,0.109244,0.031042,3.519208,732,Original Gangstas (1996),Crime,original gangstas 1996
580,0.184874,0.053215,3.474090,588,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical,aladdin 1992
159,0.142857,0.042129,3.390977,161,Crimson Tide (1995),Drama|Thriller|War,crimson tide 1995


9.PUTTING ALL OF THIS INTO A FUNCTION

In [ ]:
def find_similar_movies(movie_id):

    similar_users = ratings[(ratings["movieId"] == movie_id)& (ratings["rating"] > 4)]["userId"].unique()
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) # this is finding recommendation similar to us
    & (ratings["rating"] > 4)]["movieId"]

    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)# adjusting the per of recommendation
    similar_user_recs = similar_user_recs[similar_user_recs > .10]

    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]# this is finding common recommendation among all of the users
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())

    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)# concatinating
    rec_percentages.columns = ["similar", "all"]

    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]# generating score

    rec_percentages=rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres"]]  #sorting and returning our merge data

10.CREATING AN INTERACTIVE RECOMMENDATION WIDGET

In [ ]:
movie_name_input = widgets.Text(    # creating a widget
    value='Toy Story',
    description = "Movie Title :",  # movie titled will be entered here
    disabled=False   # it will be enabled
)

recommendation_list=widgets.Output()

def on_type(data):
  with recommendation_list:
    recommendation_list.clear_output()# removes old output
    title=data["new"]
    if len(title)>5:
      results=search(title)
      movie_id=results.iloc[0]["movieId"]
      display(find_similar_movies(movie_id))

movie_name_input.observe(on_type,names="value")
display(movie_name_input, recommendation_list)

Text(value='Toy Story', description='Movie Title :')

Output()